In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data= pd.read_csv('/kaggle/input/restaurant-revenue-prediction/train.csv.zip')
data

In [ ]:
data['Open_date'] = data['Open Date']

In [ ]:
data.dtypes

In [ ]:
data.isnull().sum()

In [ ]:
data.iloc[:,1].unique()

In [ ]:
import pandas as pd
data.Open_date = pd.to_datetime(data.Open_date,dayfirst = False)

In [ ]:
data[['year']] = pd.DataFrame(data.Open_date.dt.year)
data[['day']] = pd.DataFrame(data.Open_date.dt.day)
data[['month']] = pd.DataFrame(data.Open_date.dt.month)

In [ ]:
data.head()

In [ ]:
data.dtypes

In [ ]:
data['Revenue'] = data['revenue']

In [ ]:
data.drop(columns = ['Id','Open Date','Open_date','revenue','City','Type'],inplace = True)

In [ ]:
import seaborn as sns
sns.boxplot(data.P5)

In [ ]:
data['P5'].describe()

In [ ]:
data[(data.P5>data.P5.quantile(0.09))|(data.P5<data.P5.quantile(0.095))]

In [ ]:
import seaborn as sns
sns.boxplot(data.P7)

In [ ]:
data[(data.P7>data.P5.quantile(0.05))|(data.P5<data.P5.quantile(0.095))]

In [ ]:
import seaborn as sns
sns.distplot(data.Revenue)

In [ ]:
from sklearn.preprocessing import PowerTransformer
p = PowerTransformer(method = 'box-cox')
data['Revenue'] = p.fit_transform(data[['Revenue']])
sns.distplot(data.Revenue)

In [ ]:
data.head()

In [ ]:
x = data.iloc[:,0:41]
y = data['Revenue']

In [ ]:
x

In [ ]:
x.iloc[:,1].unique()

In [ ]:
nom =[0]
num = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,39,40]
#num = x[1:41]

In [ ]:
from sklearn.preprocessing import StandardScaler,OneHotEncoder,PowerTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn import set_config
trans = make_column_transformer((OneHotEncoder(sparse = False),nom),
                              # (OrdinalEncoder(),ord_col),
                               (PowerTransformer(),num),
                               #(Binarizer(),bin_col),
                               remainder = 'passthrough')
set_config(display = 'diagram')
trans

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.31) 

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
algorithm = LinearRegression()
pipe = make_pipeline(trans,algorithm)

In [ ]:
pipe.fit(x_train,y_train)

In [ ]:
pred = pipe.predict(x_test)
pred

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(pred,y_test)